In [1]:
import configparser
from datetime import datetime
import os

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, dayofweek, hour, weekofyear, date_format, from_unixtime
import pandas as pd

In [2]:
config = configparser.ConfigParser()
config.read_file(open('dl.cfg'))

In [3]:
os.environ['AWS_ACCESS_KEY_ID']=config['AWS']["AWS_ACCESS_KEY_ID"]
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']["AWS_SECRET_ACCESS_KEY"]

In [4]:
spark = SparkSession \
.builder \
.config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
.getOrCreate()

In [5]:
input_data = "s3a://udacity-dend/"
output_data = "s3a://misho-udacity-bucket/datalake/"

# Getting data songs

In [32]:
# df = spark.read.json("s3a://{}:{}@udacity-dend/song_data/A/B/C/TRABCEI128F424C983.json".format(os.environ['AWS_ACCESS_KEY_ID'],os.environ['AWS_SECRET_ACCESS_KEY']))

In [39]:
df = spark.read.json("s3a://{}:{}@udacity-dend/song_data/A/A/A/*.json".format(os.environ['AWS_ACCESS_KEY_ID'],os.environ['AWS_SECRET_ACCESS_KEY']))

In [40]:
df.count()

24

In [43]:
df.limit(10).toPandas()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARLTWXK1187FB5A3F8,32.74863,"Fort Worth, TX",-97.32925,King Curtis,326.00771,1,SODREIN12A58A7F2E5,A Whiter Shade Of Pale (Live @ Fillmore West),0
1,ARIOZCU1187FB3A3DC,NaN,"Hamlet, NC",NaN,JOHN COLTRANE,220.44689,1,SOCEMJV12A6D4F7667,Giant Steps (Alternate Version_ Take 5_ Altern...,0
2,ARPFHN61187FB575F6,41.88415,"Chicago, IL",-87.63241,Lupe Fiasco,279.97995,1,SOWQTQZ12A58A7B63E,Streets On Fire (Explicit Album Version),0
3,AR5S9OB1187B9931E3,34.05349,"Los Angeles, CA",-118.24532,Bullet Boys,156.62975,1,SOMAPYF12A6D4FEC3E,All Day & All Of The Night,0
4,AR5T40Y1187B9996C6,NaN,"Lulea, Sweden",NaN,The Bear Quartet,249.31220,1,SOAPVNX12AB0187625,I Remember Nights Wide Open,1998
5,AR9OEB71187B9A97C6,NaN,"Edmonton, Alberta, Canada",NaN,Faunts,397.16526,1,SOFIUVJ12A8C13C296,Will You Tell Me Then,2005
6,ARBDJHO1252CCFA6FC,NaN,,NaN,The Band of HM Royal Marines,188.73424,1,SOBHXUU12A6D4F5F14,National Emblem (March),0
7,ARAADXM1187FB3ECDB,34.16880,"Woodland Hills, CA",-118.61092,Styles Of Beyond,67.63057,1,SOQFYBD12AB0182188,Intro,1999
8,ARZJDBC1187FB52056,27.94017,"Brandon, Florida",-82.32547,Nasty Savage,327.00036,1,SOYLILV12A8C136650,XXX,1984
9,AROSPS51187B9B481F,NaN,,NaN,Vince Guaraldi Trio,197.95546,1,SOHTCZS12A6D4FC402,The Christmas Song,1965


In [41]:
song_columns = ['song_id', 'title', 'artist_id', 'year', 'duration']
artist_columns = ['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']

In [42]:
df_song = df.select(*song_columns)
df_artist = df.select(*artist_columns)

In [43]:
df_song.show()

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SOAFBCP12A8C13CC7D|King Of Scurf (20...|ARTC1LV1187B9A4858|1972|301.40036|
|SOKTJDS12AF72A25E5|Drown In My Own T...|ARA23XO1187B9AF18F|   0|  192.522|
|SOEKAZG12AB018837E|I'll Slap Your Fa...|ARSVTNL1187B992A91|2001|129.85424|
|SOQPWCR12A6D4FB2A3|A Poor Recipe For...|AR73AIO1187B9AD57B|2005|118.07302|
|SOBRKGM12A8C139EF6|Welcome to the Pl...|ARXQBR11187B98A2CC|1985|821.05424|
|SORRNOC12AB017F52B|The Last Beat Of ...|ARSZ7L31187FB4E610|2004|337.81506|
|SOHKNRJ12A6701D1F8|        Drop of Rain|AR10USD1187B99F3F1|   0|189.57016|
|SOAPERH12A58A787DC|The One And Only ...|ARZ5H0P1187B98A1DD|   0|230.42567|
|SOSMJFC12A8C13DE0C|Is That All There...|AR1KTV21187B9ACD72|   0|343.87546|
|SOOVHYF12A8C134892|     I'll Be Waiting|ARCLYBR1187FB53913|1989|304.56118|
|SOERIDA12A6

In [44]:
df_artist.show()

+------------------+--------------------+--------------------+---------------+----------------+
|         artist_id|         artist_name|     artist_location|artist_latitude|artist_longitude|
+------------------+--------------------+--------------------+---------------+----------------+
|ARTC1LV1187B9A4858|  The Bonzo Dog Band|Goldsmith's Colle...|        51.4536|        -0.01802|
|ARA23XO1187B9AF18F|     The Smithereens|Carteret, New Jersey|       40.57885|       -74.21956|
|ARSVTNL1187B992A91|       Jonathan King|     London, England|       51.50632|        -0.12714|
|AR73AIO1187B9AD57B|   Western Addiction|   San Francisco, CA|       37.77916|      -122.42005|
|ARXQBR11187B98A2CC|Frankie Goes To H...|  Liverpool, England|           null|            null|
|ARSZ7L31187FB4E610|           Devotchka|          Denver, CO|       39.74001|      -104.99226|
|AR10USD1187B99F3F1|Tweeterfriendly M...|Burlington, Ontar...|           null|            null|
|ARZ5H0P1187B98A1DD|          Snoop Dogg

### Partition

In [38]:
df_song.write.parquet('output/songs', mode='overwrite', partitionBy=['year', 'artist_id'])

# Write data

In [35]:
df_song.write.parquet(output_data+'/songs/')

# Getting Data logs

In [36]:
df2 = spark.read.json("s3a://{}:{}@udacity-dend/log_data/2018/11/2018-11-12*.json"\
                      .format(os.environ['AWS_ACCESS_KEY_ID'],os.environ['AWS_SECRET_ACCESS_KEY']))

In [37]:
df2.count()

213

In [38]:
df2.limit(5).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Celeste,F,0,Williams,NaN,free,"Klamath Falls, OR",GET,Home,1.541078e+12,438,None,200,1541990217796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
1,Pavement,Logged In,Sylvie,F,0,Cruz,99.16036,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",PUT,NextSong,1.540266e+12,345,Mercy:The Laundromat,200,1541990258796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10
2,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Logged In,Celeste,F,1,Williams,277.15873,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,438,Horn Concerto No. 4 in E flat K495: II. Romanc...,200,1541990264796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
3,Gary Allan,Logged In,Celeste,F,2,Williams,211.22567,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,438,Nothing On But The Radio,200,1541990541796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
4,None,Logged In,Jacqueline,F,0,Lynch,NaN,paid,"Atlanta-Sandy Springs-Roswell, GA",GET,Home,1.540224e+12,389,None,200,1541990714796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",29


In [45]:
df2 = df2.filter(df2['page']=='NextSong')

In [46]:
df2.columns

['artist',
 'auth',
 'firstName',
 'gender',
 'itemInSession',
 'lastName',
 'length',
 'level',
 'location',
 'method',
 'page',
 'registration',
 'sessionId',
 'song',
 'status',
 'ts',
 'userAgent',
 'userId']

In [11]:
users_columns = ['userId', 'firstName', 'lastName', 'gender', 'level']

In [12]:
df_users = df2.select(*users_columns)

In [13]:
df_users.limit(2).toPandas()

,userId,firstName,lastName,gender,level
0,26,Ryan,Smith,M,free
1,26,Ryan,Smith,M,free


In [14]:
# timestamp

In [47]:
df2 = df2.withColumn('datetime', from_unixtime(col('ts')/1000))

In [48]:
df_time = df2.select('datetime')

In [49]:
df_time = df_time.withColumnRenamed('datetime', 'start_time')\
                 .orderBy('start_time', ascending=True)\
                 .withColumn('hour', hour(col('start_time')))\
                 .withColumn('day', dayofmonth(col('start_time')))\
                 .withColumn('week', weekofyear(col('start_time')))\
                 .withColumn('month', month(col('start_time')))\
                 .withColumn('year', year(col('start_time')))\
                 .withColumn('weekday', dayofweek(col('start_time')))

In [50]:
df_time.show()

+-------------------+----+---+----+-----+----+-------+
|         start_time|hour|day|week|month|year|weekday|
+-------------------+----+---+----+-----+----+-------+
|2018-11-12 02:37:38|   2| 12|  46|   11|2018|      2|
|2018-11-12 02:37:44|   2| 12|  46|   11|2018|      2|
|2018-11-12 02:42:21|   2| 12|  46|   11|2018|      2|
|2018-11-12 02:45:52|   2| 12|  46|   11|2018|      2|
|2018-11-12 02:47:22|   2| 12|  46|   11|2018|      2|
|2018-11-12 02:50:21|   2| 12|  46|   11|2018|      2|
|2018-11-12 02:54:26|   2| 12|  46|   11|2018|      2|
|2018-11-12 02:57:12|   2| 12|  46|   11|2018|      2|
|2018-11-12 03:00:48|   3| 12|  46|   11|2018|      2|
|2018-11-12 03:03:24|   3| 12|  46|   11|2018|      2|
|2018-11-12 03:36:28|   3| 12|  46|   11|2018|      2|
|2018-11-12 06:13:27|   6| 12|  46|   11|2018|      2|
|2018-11-12 08:50:44|   8| 12|  46|   11|2018|      2|
|2018-11-12 08:54:58|   8| 12|  46|   11|2018|      2|
|2018-11-12 08:58:41|   8| 12|  46|   11|2018|      2|
|2018-11-1

In [81]:
# df_time.write.parquet('output/time', mode='overwrite', partitionBy=['year', 'month'])

# songplays

In [18]:
basePath= output_data+'/songs/'
song_df = spark.read.option("basePath",basePath).parquet(output_data+'/songs/*')

In [51]:
# song_df.write.parquet('output/song_df')

In [19]:
song_df.show()

+------------------+--------------------+---------+----+------------------+
|           song_id|               title| duration|year|         artist_id|
+------------------+--------------------+---------+----+------------------+
|SOBTCUI12A8AE48B70|Faust: Ballet Mus...| 94.56281|   0|ARSUVLW12454A4C8B8|
|SOVNKJI12A8C13CB0D|Take It To Da Hou...|227.10812|2001|ARWUNH81187FB4A3E0|
|SOYVBGZ12A6D4F92A8|Piano Sonata No. ...|221.70077|   0|ARLRWBW1242077EB29|
|SODBHKO12A58A77F36|Fingers Of Love (...|335.93424|   0|ARKGS2Z1187FB494B5|
|SOGXFIF12A58A78CC4|Hanging On (Mediu...|204.06812|   0|AR5LZJD1187FB4C5E5|
|SOZCRVP12A81C21F40|Welcome To The Do...| 46.94159|2008|AR4503S1187FB43199|
|SOOBEML12A8C138C91|Johnny Leary's Po...|  197.642|   0|ARP4O0W1187FB5A06B|
|SOUOPFM12AB0185809|You'd Be So Nice ...|405.41995|   0|ARSXDJO1269FCD9405|
|SOVJXVJ12A8C13517D|Where The Thunder...|298.84036|   0|ARCCRTI11F4C845308|
|SOKTJDS12AF72A25E5|Drown In My Own T...|  192.522|   0|ARA23XO1187B9AF18F|
|SOHHANU12A5

In [52]:
df2.limit(1).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,datetime
0,Pavement,Logged In,Sylvie,F,0,Cruz,99.16036,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",PUT,NextSong,1.540266e+12,345,Mercy:The Laundromat,200,1541990258796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10,2018-11-12 02:37:38


In [53]:
df_song.limit(1).toPandas()

,song_id,title,artist_id,year,duration
0,SOAFBCP12A8C13CC7D,King Of Scurf (2007 Digital Remaster),ARTC1LV1187B9A4858,1972,301.40036


In [54]:
# ['artist',
#  'auth',
#  'firstName',
#  'gender',
#  'itemInSession',
#  'lastName',
#  'length',
#  'level',
#  'location',
#  'method',
#  'page',
#  'registration',
#  'sessionId',
#  'song',
#  'status',
#  'ts',
#  'userAgent',
#  'userId']
dfs = df2.select('datetime', 'userId', 'level', 'song', 'artist', 'sessionId', 'location', 'userAgent')

In [55]:
dfs.limit(1).toPandas()

,datetime,userId,level,song,artist,sessionId,location,userAgent
0,2018-11-12 02:37:38,10,free,Mercy:The Laundromat,Pavement,345,"Washington-Arlington-Alexandria, DC-VA-MD-WV","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."


In [56]:
dfs.count()

166

In [77]:
songplays_table = dfs.join(df_song, dfs.song == df_song.title, how='left')

In [78]:
songplays_table.count()

166

In [88]:
songplays_table.limit(1).toPandas()

,start_time,user_id,level,session_id,location,user_agent,song_id,artist_id
0,2018-11-12 02:37:38,10,free,345,"Washington-Arlington-Alexandria, DC-VA-MD-WV","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",None,None


In [82]:
songplays_table = songplays_table.drop('song', 'artist', 'title', 'year', 'duration')

In [87]:
columns_name = ['start_time', 'user_id', 'level', 'session_id', 'location', 'user_agent', 'song_id', 'artist_id']
songplays_table = songplays_table.toDF(*columns_name)

In [92]:
songplays_table = songplays_table.withColumn('month', month(col('start_time')))\
                                 .withColumn('year', year(col('start_time')))

In [93]:
songplays_table.limit(1).toPandas()

,start_time,user_id,level,session_id,location,user_agent,song_id,artist_id,month,year
0,2018-11-12 02:37:38,10,free,345,"Washington-Arlington-Alexandria, DC-VA-MD-WV","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",None,None,11,2018


In [94]:
songplays_table.write.parquet(output_data+'/songplays', 
                              mode='overwrite', 
                              partitionBy=['year', 'month'])